# OPN 10: Task a asynchronní programování
---

Obsah:
- Task
- Klíčové slova async a await

## Task

Pomocí třídy `Task` můžeme spouštět typicky asynchronně jednu operaci (metodu) bez návratové hodnoty. 

V následujícím příkladu spustíme provádění metody Metoda pomocí statické metody `Task.Run`, tato metoda nám vrátí proměnnou typu `Task`, krerá popisuje již spuštěnou operaci běžící typicky v jiném vlákně. 

V následujícím příkladu nejprve pomocí příkazu `Task.Run(DlouhoBeziciMetoda);` spustíme provádění metody `DlouhoBeziciMetoda`, která vypisuje text na konzoli v intervalu jedné sekundy. A poté začneme souběžně vypisovat v hlavním(main) vlákně ve stejném intervalu také na konzoli. Zápis na konzoli je možný a bezpečný z obou vláken, protože konzole zápis a čtení synchronizuje [(Console I/O Streams. Microsoft Docs. 2022)](https://docs.microsoft.com/en-us/dotnet/api/system.console?view=net-6.0#console-io-streams). 

Příkaz `task.Wait()` na konci čeká na dokončení tasku tak, aby se nám program předčasně neukončil před jeho dokončením.

In [24]:
void DlouhoBeziciMetoda()
{
    for (int i = 0; i < 5; i++)
    {
        Console.WriteLine($"Metoda: {i}");
        System.Threading.Thread.Sleep(1000);
    }
}

Task task = Task.Run(DlouhoBeziciMetoda);

for (int i = 0; i < 8; i++)
{
    Console.WriteLine($"Main: {i}");
    System.Threading.Thread.Sleep(1000);
}

task.Wait();

Main: 0
Metoda: 0
Main: 1
Metoda: 1
Main: 2
Metoda: 2
Main: 3
Metoda: 3
Main: 4
Metoda: 4
Main: 5
Main: 6
Main: 7


Zkuste si v předcházejícím příkladu zvolit kratší interval pro výpis v hlavním vlákně a případně odstranit poslední příkaz task.Wait();

Pokud bychom použili příkaz `task.Wait()` hned po spuštění tasku, tak se běh hlavního vlákna pozastaví dokud nebude task dokončen a výsledkem by bylo, že výpisy neproběhnou synchronně.

In [27]:
void DlouhoBeziciMetoda()
{
    for (int i = 0; i < 5; i++)
    {
        Console.WriteLine($"Metoda: {i}");
        System.Threading.Thread.Sleep(1000);
    }
}

Task task = Task.Run(DlouhoBeziciMetoda);

task.Wait();

for (int i = 0; i < 8; i++)
{
    Console.WriteLine($"Main: {i}");
    System.Threading.Thread.Sleep(1000);
}


Metoda: 0
Metoda: 1
Metoda: 2
Metoda: 3
Metoda: 4
Main: 0
Main: 1
Main: 2
Main: 3
Main: 4
Main: 5
Main: 6
Main: 7


## `Task<T>` s návratovou hodnotou

Pokud metoda vrací hodnotu, tak můžeme využít generický typ `Task<T>`. 

V následujícím příkladu vrací `TDlouhoBeziciMetoda` náhodné číslo po dokončení simulovaného dlouhého výpočtu. Výsledek získáme pomocí příkazu `task.Result`, který opět pozastaví hlavní vlákno, dokud se task nedokončí.

In [30]:
int DlouhoBeziciMetoda()
{
    for (int i = 0; i < 5; i++)
    {
        Console.WriteLine($"Metoda: {i}");
        System.Threading.Thread.Sleep(1000);
    }

    return Random.Shared.Next(0, 100);
}

Task<int> task = Task.Run(DlouhoBeziciMetoda);

int vysledek = task.Result;

Console.WriteLine($"vysledek: {vysledek}");

for (int i = 0; i < 8; i++)
{
    Console.WriteLine($"Main: {i}");
    System.Threading.Thread.Sleep(1000);
}

Metoda: 0
Metoda: 1
Metoda: 2
Metoda: 3
Metoda: 4
vysledek: 50
Main: 0
Main: 1
Main: 2
Main: 3
Main: 4
Main: 5
Main: 6
Main: 7


## Klíčové slova async a await

Pomocí klíčových slov `async` a `await` můžeme jednoduše synchroynizovat asynchronní operace a kód zůstal přehledný a mohli jsme používat vyjímky stejným způsobem jako v běžném kódu. Například v programu často chceme stáhnout v seznam produktů v jiném vlákně, tak aby se nám nezablokovalo hlavní vlákno klientské aplikaci a po dokončení stažení produků, chceme tyto produkty zobrazit.

Proto, abychom mohli demonstrovat použití klíčového slova `await`. Tak si vytvoříme složitější příklad. Spuštění tasku je přesunuté přímo do metody `DlouhoBeziciMetodaAsync`, která vrací přímo typ `Task<int>`. Podle konvence má mít název metody, která vrací Task příponu *Async*. Potom máme druhou metodu, která čeká na dokončení metody `DlouhoBeziciMetodaAsync` a pak zobrazí výsledek výpočtu.

Nejprve si ale ukážeme příklad bez async, kde používáme příkaz `task.Result`, což znamená že se se hlavní vlákno pozastaví a kód se neprovádí souběžně. 

In [31]:
Task<int> DlouhoBeziciMetodaAsync()
{
    return Task.Run(() =>
    {
        for (int i = 0; i < 5; i++)
        {
            Console.WriteLine($"Metoda: {i}");
            System.Threading.Thread.Sleep(1000);
        }

        return Random.Shared.Next(0, 100); 
    });
}

void MetodaCekajiciNaVysledek()
{
    Task<int> task = DlouhoBeziciMetodaAsync();
    
    int vysledek = task.Result;

    Console.WriteLine($"vysledek: {vysledek}");
}

MetodaCekajiciNaVysledek();

for (int i = 0; i < 8; i++)
{
    Console.WriteLine($"Main: {i}");
    System.Threading.Thread.Sleep(1000);
}

Console.WriteLine("Konec programu");

Metoda: 0
Metoda: 1
Metoda: 2
Metoda: 3
Metoda: 4
vysledek: 20
Main: 0
Main: 1
Main: 2
Main: 3
Main: 4
Main: 5
Main: 6
Main: 7
Konec programu


Nyní použijeme klíčové slovo `await`, díky čemuž se `DlouhoBeziciMetodaAsync` provádí souběžně s kódem v hlavním vlákně a po jejím dokončení se hlavní vlákno přeruší, provede se kód za příkazem `await` a pak zase pokračuje hlavní vlákno. Klíčové slovo `await` můžeme použít jen v metodě označené klíčovým slovem `async`.

In [33]:
Task<int> DlouhoBeziciMetodaAsync()
{
    return Task.Run(() =>
    {
        for (int i = 0; i < 5; i++)
        {
            Console.WriteLine($"Metoda: {i}");
            System.Threading.Thread.Sleep(1000);
        }

        return Random.Shared.Next(0, 100); 
    });
}

async void MetodaCekajiciNaVysledek()
{
    Task<int> task = DlouhoBeziciMetodaAsync();
    
    int vysledek = await task;

    Console.WriteLine($"vysledek: {vysledek}");
}

MetodaCekajiciNaVysledek();

for (int i = 0; i < 8; i++)
{
    Console.WriteLine($"Main: {i}");
    System.Threading.Thread.Sleep(1000);
}

Console.WriteLine("Konec programu");

Main: 0
Metoda: 0
Metoda: 1
Main: 1
Metoda: 2
Main: 2
Main: 3
Metoda: 3
Main: 4
Metoda: 4
Main: 5
vysledek: 85
Main: 6
Main: 7
Konec programu


In [ ]:
int DlouhoBeziciMetoda(bool exception)
{
    for (int i = 0; i < 5; i++)
    {
        Console.WriteLine($"Metoda: {i}");
        System.Threading.Thread.Sleep(1000);

        if(exception && Random.Shared.Next() % 5 == 0)
        {
            throw new Exception("Nahodna vyjimka");
        } 
    }

    return 5;
}


In [24]:
int DlouhoBeziciMetoda(bool exception)
{
    for (int i = 0; i < 5; i++)
    {
        Console.WriteLine($"Metoda: {i}");
        System.Threading.Thread.Sleep(1000);

        if(exception && Random.Shared.Next() % 5 == 0)
        {
            throw new Exception("Nahodna vyjimka");
        } 
    }

    return 5;
}

void MetodaCekajiciNaVysledek()
{
    try
    {
        int vysledek = DlouhoBeziciMetoda(exception: true);
        Console.WriteLine($"vysledek: {vysledek}");
    }
    catch(Exception ex)
    {
        Console.WriteLine($"{ex.GetType()}: {ex.Message}");
    }
}

MetodaCekajiciNaVysledek();

for (int i = 0; i < 5; i++)
{
    Console.WriteLine($"Main: {i}");
    System.Threading.Thread.Sleep(1000);
}

Console.WriteLine("Konec programu");

Metoda: 0
Metoda: 1
Metoda: 2
Metoda: 3
Metoda: 4
System.Exception: Nahodna vyjimka
Main: 0
Main: 1
Main: 2
Main: 3
Main: 4
Konec programu


In [25]:
Task<int> DlouhoBeziciMetodaAsync(bool exception)
{
    return Task.Run(() =>
    {
        for (int i = 0; i < 5; i++)
        {
            Console.WriteLine($"Metoda: {i}");
            System.Threading.Thread.Sleep(1000);

            if(exception && Random.Shared.Next() % 5 == 0)
            {
                throw new Exception("Nahodna vyjimka");
            } 
        }

        return 5;
    });
}

void MetodaCekajiciNaVysledek()
{
    try
    {
        Task<int> task = DlouhoBeziciMetodaAsync(exception: true);
        
        task.Wait();

        int vysledek = task.Result;

        Console.WriteLine($"vysledek: {vysledek}");
    }
    catch(Exception ex) // https://learn.microsoft.com/en-us/dotnet/standard/parallel-programming/exception-handling-task-parallel-library
    {
        Console.WriteLine($"{ex.GetType()}: {ex.Message}");
    }
}

MetodaCekajiciNaVysledek();

for (int i = 0; i < 5; i++)
{
    Console.WriteLine($"Main: {i}");
    System.Threading.Thread.Sleep(1000);
}

Console.WriteLine("Konec programu");

Metoda: 0
Metoda: 1
Metoda: 2
Metoda: 3
Metoda: 4
System.AggregateException: One or more errors occurred. (Nahodna vyjimka)
Main: 0
Main: 1
Main: 2
Main: 3
Main: 4
Konec programu


In [32]:
Task<int> DlouhoBeziciMetodaAsync(bool exception)
{
    return Task.Run(() =>
    {
        for (int i = 0; i < 5; i++)
        {
            Console.WriteLine($"Metoda: {i}");
            System.Threading.Thread.Sleep(1000);

            if(exception && Random.Shared.Next() % 5 == 0)
            {
                throw new Exception("Nahodna vyjimka");
            } 
        }

        return 5;
    });
}

void MetodaCekajiciNaVysledek()
{
    try
    {
        Task<int> task = DlouhoBeziciMetodaAsync(exception: true);
        
        task.ContinueWith(task => 
        {
            if(task.Status == TaskStatus.RanToCompletion)
            {
                Console.WriteLine($"vysledek: {task.Result}");
            }
            else
            {
                if(task.Status == TaskStatus.Faulted)
                {
                    Console.WriteLine($"Chyba: {task.Exception.Message}");
                }
            }
        });
    }
    catch(Exception ex)
    {
        Console.WriteLine($"{ex.GetType()}: {ex.Message}");
    }
}

MetodaCekajiciNaVysledek();

for (int i = 0; i < 5; i++)
{
    Console.WriteLine($"Main: {i}");
    System.Threading.Thread.Sleep(1000);
}

Console.WriteLine("Konec programu");

Main: 0
Metoda: 0
Main: 1
Chyba: One or more errors occurred. (Nahodna vyjimka)
Main: 2
Main: 3
Main: 4
Konec programu


In [33]:
Task<int> DlouhoBeziciMetodaAsync(bool exception)
{
    return Task.Run(() =>
    {
        for (int i = 0; i < 5; i++)
        {
            Console.WriteLine($"Metoda: {i}");
            System.Threading.Thread.Sleep(1000);

            if(exception && Random.Shared.Next() % 5 == 0)
            {
                throw new Exception("Nahodna vyjimka");
            } 
        }

        return 5;
    });
}

async void MetodaCekajiciNaVysledek()
{
    try
    {
        Task<int> task  = DlouhoBeziciMetodaAsync(exception: true);

        int vysledek = await task;

        Console.WriteLine($"vysledek: {vysledek}");
    }
    catch(Exception ex)
    {
        Console.WriteLine($"{ex.GetType()}: {ex.Message}");
    }
}

MetodaCekajiciNaVysledek();

for (int i = 0; i < 5; i++)
{
    Console.WriteLine($"Main: {i}");
    System.Threading.Thread.Sleep(1000);
}

Console.WriteLine("Konec programu");

Main: 0
Metoda: 0
Main: 1
System.Exception: Nahodna vyjimka
Main: 2
Main: 3
Main: 4
Konec programu


Příkaz `await task;` nepozastaví běh programu, ale jen přeruší provádění metody `Main` a mezití běží ostatní operace a po dokončení metody `Metoda` bude v provádění metody `Main` pokračovat. 

In [ ]:
int DlouhoBeziciMetoda()
{
    for (int i = 0; i < 10; i++)
    {
        Console.WriteLine($"Metoda: {i}");
        System.Threading.Thread.Sleep(1000);
    }

    return 5;
}

void MetodaCekajiciNaVysledek()
{
    int vysledek = DlouhoBeziciMetoda();
    Console.WriteLine($"vysledek: {vysledek}");
}

Task task = Task.Run(DlouhoBeziciMetoda);

Console.WriteLine("Task spusten");

task.Wait();

for (int i = 0; i < 10; i++)
{
    Console.WriteLine($"Main: {i}");
    System.Threading.Thread.Sleep(1000);
}

Console.WriteLine("Konec programu");

In [ ]:
void DlouhoBeziciMetoda()
{
    for (int i = 0; i < 10; i++)
    {
        Console.WriteLine($"Metoda: {i}");
        System.Threading.Thread.Sleep(400);
    }
}

void MetodaCekajiciNaVysledek()
{
    
}

Task task = Task.Run(DlouhoBeziciMetoda);
Console.WriteLine("Task spusten");
await task;

Console.WriteLine("Konec programu");
